# Yao.jl
A flexible whitebox Quantum Circuit algorithm design toolkit.

## Feel
We have a register (i.e. a quantum state) $|\psi\rangle$ represented as a vector.

In [1]:
using Compat
using Compat.Test
using Yao
using Yao.Blocks
using Yao.Intrinsics
reg = register(bit"000") + register(bit"101") |> normalize!
reg |> statevec

8-element Array{Complex{Float64},1}:
 0.707107+0.0im
      0.0+0.0im
      0.0+0.0im
      0.0+0.0im
      0.0+0.0im
 0.707107+0.0im
      0.0+0.0im
      0.0+0.0im

You may have noticed, our register orders like $|b_n\rangle \ldots |b_2\rangle|b_1\rangle$, whose Hilbert space is defined on basis:

In [2]:
# indexing order
display(basis(3) |> collect |> bitarray(3) .|> Int)
# numbering order
display(bin.(basis(3), 3))

3×8 Array{Int64,2}:
 0  1  0  1  0  1  0  1
 0  0  1  1  0  0  1  1
 0  0  0  0  1  1  1  1

8-element Array{String,1}:
 "000"
 "001"
 "010"
 "011"
 "100"
 "101"
 "110"
 "111"

In [3]:
with!(reg) = reg
with(reg) = copy(reg)
import Base: |>
|>(reg::AbstractRegister, blk::AbstractBlock) = apply!(reg, blk)
import Yao.Intrinsics.bitarray

An predefined NOT gate $X$, is represented as a [generalized permute matrix](https://en.wikipedia.org/wiki/Generalized_permutation_matrix).

In [4]:
# X is a single qubit gate
display(mat(X))

# We want to focus on the sencond bit to apply X, then cancel the focus operation.
# All we need is focus.
F = Focus(3)
with(reg) |> F(2) |> X |> F(nothing) |> statevec

2×2 Yao.LuxurySparse.PermMatrix{Complex{Float64},Int64}:
    0       1.0+0.0im
 1.0+0.0im     0     

8-element Array{Complex{Float64},1}:
      0.0+0.0im
      0.0+0.0im
 0.707107+0.0im
      0.0+0.0im
      0.0+0.0im
      0.0+0.0im
      0.0+0.0im
 0.707107+0.0im

### What Happened Here?
1. Focus on the 2nd bit, so blocks see the 2nd bit only,
2. Apply X on this active bit,
3. Focus(nothing) to roll back all focus operations.

In [33]:
F = Focus(nqubits(reg))
with(reg) |> F(2,3) |> MEASURE |> F(nothing) |> statevec

8-element Array{Complex{Float64},1}:
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im
 1.0+0.0im
 0.0+0.0im
 0.0+0.0im

In [34]:
MEASURE.result |> bitarray(2)

2×1 BitArray{2}:
 false
  true

In [35]:
with(reg) |> X

LoadError: [91mDimensionMismatch("")[39m

We don't need fix

`chain`'s behavior likes a `Vector`, itself is complete to describe any circuit.

Apply this operator on the 2nd qubit, means calculating $I_2\otimes X\otimes I_2 |\psi\rangle$ 

In [7]:
IXI = kron(3, 2=>X) # size and pair

Total: 3, DataType: Complex{Float64}
kron
└─ 2=>X gate


Notice here, `kron(2, 2=>X)` returns a `KronBlock` instance, `X` is a predefined `XGate` instance, both are `MatrixBlocks`.

`MatrixBlock` implements `mat` interface.

In [8]:
using Yao.Blocks
@assert X isa MatrixBlock
@assert kron(2, 2=>X) isa MatrixBlock
@assert issubtype(MatrixBlock, AbstractBlock)
mat(IXI)

8×8 Yao.LuxurySparse.PermMatrix{Complex{Float64},Int64}:
    0          0       1.0+0.0im  …     0          0          0     
    0          0          0             0          0          0     
 1.0+0.0im     0          0             0          0          0     
    0       1.0+0.0im     0             0          0          0     
    0          0          0             0       1.0+0.0im     0     
    0          0          0       …     0          0       1.0+0.0im
    0          0          0             0          0          0     
    0          0          0          1.0+0.0im     0          0     

In [9]:
reg |> IXI
statevec(reg)

8-element Array{Complex{Float64},1}:
      0.0+0.0im
      0.0+0.0im
 0.707107+0.0im
      0.0+0.0im
      0.0+0.0im
 0.707107+0.0im
      0.0+0.0im
      0.0+0.0im

Then we want to build a larger circuit $C = (H\otimes H\otimes H)(I_2\otimes X\otimes I_2)$
![circuit_xhhh.png](attachment:circuit_xhhh.png)

In [17]:
bigreg = rand_state(5)
HHH = kron(3, (i=>H for i in 1:3)...)
circuit = chain(IXI, HHH)
reg2 = with(bigreg) |> Concentrator{5}(circuit, [5,1,2]) |> statevec

32-element Array{Complex{Float64},1}:
   -0.037869+0.126653im 
   0.0442067+0.126734im 
  -0.0625266-0.154198im 
   0.0544712-0.288671im 
  -0.0126715-0.0760913im
    0.053274+0.15601im  
     0.11723+0.0539733im
   0.0398053-0.0638877im
  -0.0244146-0.0316419im
   -0.159306+0.0564486im
    0.120083-0.0630978im
    0.133407-0.0613283im
   0.0829781-0.240048im 
            ⋮           
  -0.0405701-0.111963im 
   -0.147527-0.0327301im
   0.0219751-0.0884933im
   -0.117001+0.0570855im
   -0.259686-0.0189936im
 0.000821339+0.0439073im
   0.0592133+0.145806im 
  -0.0351971+0.261564im 
   -0.247322-0.0985339im
   0.0816123-0.272345im 
    -0.21225-0.227346im 
     -0.1398+0.145889im 

In [15]:
using Yao.Intrinsics
res = measure(reg2)

LoadError: [91mUndefVarError: reg2 not defined[39m

In [84]:
bitarray(measure(reg2, 10), num_bit=2)

2×10 SubArray{Bool,2,BitArray{2},Tuple{UnitRange{Int64},Base.Slice{Base.OneTo{Int64}}},false}:
  true  false  false  false  false   true   true  true  true  false
 false   true   true   true   true  false  false  true  true  false

# Imaging

## Everything is Block
* `Primitive Block` is a block that does not have sub-blocks.
* `Composite Block` is a block that is constructed using sub-blocks.

In [31]:
bigger = kron(10, 2=>IXI, [i=>X for i = 5:6]..., 8=>phase(0.3))

Total: 10, DataType: Complex{Float64}
kron
├─ 2=>kron
│  └─ 2=>X gate
├─ 5=>X gate
├─ 6=>X gate
└─ 8=>Global Phase Gate:0.3


In [30]:
repeat(3, bigger)

Total: 3, DataType: Complex{Float64}
repeat on (1, 2, 3)
└─ kron
   ├─ 2=>kron
   │  └─ 2=>X gate
   ├─ 5=>X gate
   ├─ 6=>X gate
   └─ 8=>Global Phase Gate:0.3


## Luxury Sparse
In our `LuxurySparse` module, we implemented some high performance matrix types

* `PermMatrix`
* `Identity`
* `Diagonal` (extended its `kron` and `multiply` operations)

Making matrix operations more efficient.

In [10]:
using Yao
using Yao.Blocks
import Yao.Blocks:mat, ControlBlock, XGate
function mat(ctrl::Yao.Blocks.ControlBlock{2, <:XGate, 1, Bool})
    println("calling controlled_U1")
    #controlled_U1(N, mat(ctrl.block), [ctrl.ctrl_qubits...], [ctrl.vals...], ctrl.addr)
end

cb = ControlBlock{2}((2,), X, 1)

typeof(cb)

Yao.Blocks.ControlBlock{2,Yao.Blocks.XGate{Complex{Float64}},1,Bool}

In [11]:
linop2dense(r->yapply!(r,[1,2]), 2)

LoadError: [91mUndefVarError: linop2dense not defined[39m

In [12]:
mat(cb)

calling controlled_U1


In [13]:
XGate <: MatrixBlock{1}

true

In [14]:
matvec

LoadError: [91mUndefVarError: matvec not defined[39m